This notebook is a continuation of the work in the titanic-eda.ipynb.  The previous notebook did exploratory data analysis on the titanic survivors dataset and explored relationships between variables found in that set.  While exploring
we found a large number of missing values that should be filled in before any relationships are modeled.  To start I will display the percentage of missing values in each category so I know where I will be focused.

In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", None, "display.max_columns", None)

test_data = pd.read_csv ('data/test.csv')
train_data = pd.read_csv("data/train.csv")

combined=train_data.append(test_data)

##We discussed the below command in detail the data exploration notebook
print('Missing values Percentage: \n\n', round (combined.isnull().sum().sort_values(ascending=False)/len(combined)*100,1))

Missing values Percentage: 

 Cabin          77.5
Survived       31.9
Age            20.1
Embarked        0.2
Fare            0.1
Ticket          0.0
Parch           0.0
SibSp           0.0
Sex             0.0
Name            0.0
Pclass          0.0
PassengerId     0.0
dtype: float64


So many cabin values are missing that attempting to fill them in would create a large amount of noise in our final model.  A few embarked values and one fare are missing but the most pressing category is the age column.  Age
is a strong indecator of survival, especally seeing as children are likely to survive and elderly are unlikely to survive.  A simple approach would be to use the mean or median as an age filler, but in the EDA notebook we found
grouping by sex and class is a good predictor of age.  Additionally we can use title to determine if a passenger is a young boy with a missing age since their title will be "master".  Since that is the most specific piece we can start with that.

In [2]:
display(combined[(combined.Age.isnull()) & (combined.Name.str.contains('Master'))])
#This simply displays all passengers with the master title who are missing age values

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
65,66,1.0,3,"Moubarek, Master. Gerios",male,NaN,1,1,2661,15.2458,NaN,C
159,160,0.0,3,"Sage, Master. Thomas Henry",male,NaN,8,2,CA. 2343,69.5500,NaN,S
176,177,0.0,3,"Lefebre, Master. Henry Forbes",male,NaN,3,1,4133,25.4667,NaN,S
709,710,1.0,3,"Moubarek, Master. Halim Gonios (""William George"")",male,NaN,1,1,2661,15.2458,NaN,C
244,1136,NaN,3,"Johnston, Master. William Arthur Willie""""",male,NaN,1,2,W./C. 6607,23.4500,NaN,S
339,1231,NaN,3,"Betros, Master. Seman",male,NaN,0,0,2622,7.2292,NaN,C
344,1236,NaN,3,"van Billiard, Master. James William",male,NaN,1,1,A/5. 851,14.5000,NaN,S
417,1309,NaN,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


This returned 8 children.  If we had taken the mean approach we would have assigned each of them as 30 years old.  Since children as <14yrs old that would have been a significant oversight.  In order to fill these in lets get the mean for all
other masters on board.

In [3]:
print(train_data[train_data.Name.str.contains('Master')]['Age'].mean())

4.574166666666667


While observing the above table I noticed there is an entry with 0 for the Parch column which means they are not with parents.  Maybe they were with relatives or a friend, so it's safe to assume they are likely older than the mean of 5.
They will be assigned 14 because I doubt a child less than 10 would be onboard without parents and siblings.

In [4]:
test_data.loc[test_data.PassengerId==1231,'Age']=14

In [5]:
train_data['Title'], test_data['Title'] = [df.Name.str.extract (' ([A-Za-z]+)\.', expand=False) for df in [train_data, test_data]]

The previous line is a list comp that extracts the title from each name and stores them as a new column in test and train frames.  Now that we have this we can print the average age across title and pclass.

In [6]:
train_data.groupby(['Title', 'Pclass'])['Age'].agg(['mean', 'count'])

mean  count
Title    Pclass                  
Capt     1       70.000000      1
Col      1       58.000000      2
Countess 1       33.000000      1
Don      1       40.000000      1
Dr       1       43.750000      4
         2       38.500000      2
Jonkheer 1       38.000000      1
Lady     1       48.000000      1
Major    1       48.500000      2
Master   1        5.306667      3
         2        2.258889      9
         3        5.350833     24
Miss     1       30.000000     45
         2       22.390625     32
         3       16.123188     69
Mlle     1       24.000000      2
Mme      1       24.000000      1
Mr       1       41.580460     87
         2       32.768293     82
         3       28.724891    229
Mrs      1       40.882353     34
         2       33.682927     41
         3       33.515152     33
Ms       2       28.000000      1
Rev      2       43.166667      6
Sir      1       49.000000      1

This seems like too many titles so I will compress this a little. Jonkheer, Don, Dr, Countess, Sir, and Rev will be classified as royalty because they are close in age.

In [7]:
TitleDict = {"Capt": "Officer","Col": "Officer","Major": "Officer","Jonkheer": "Royalty", \
             "Don": "Royalty", "Sir" : "Royalty","Dr": "Royalty","Rev": "Royalty", \
             "Countess":"Royalty", "Mme": "Mrs", "Mlle": "Miss", "Ms": "Mrs","Mr" : "Mr", \
             "Mrs" : "Mrs","Miss" : "Miss","Master" : "Master","Lady" : "Royalty"}

In [8]:
#Lets reset the groups and display them

In [9]:
train_data['Title'], test_data['Title'] = [df.Title.map(TitleDict) for df in [train_data, test_data]]

##Let us now reprint the groups
train_data.groupby(['Title', 'Pclass'])['Age'].agg(['mean', 'count'])

mean  count
Title   Pclass                  
Master  1        5.306667      3
        2        2.258889      9
        3        5.350833     24
Miss    1       29.744681     47
        2       22.390625     32
        3       16.123188     69
Mr      1       41.580460     87
        2       32.768293     82
        3       28.724891    229
Mrs     1       40.400000     35
        2       33.547619     42
        3       33.515152     33
Officer 1       56.600000      5
Royalty 1       42.555556      9
        2       42.000000      8

Since test was not used in creating this dictionary lets do a final check that all titles are covered.

In [10]:
combined=train_data.append(test_data)
display(train_data[train_data.Title.isnull()])
display(test_data[test_data.Title.isnull()])

##There is Dona which is royalty which is not covered in test_data. Update the same
test_data.at[414,'Title'] = 'Royalty'

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9,C105,C,NaN


We can easily identify male children due to the master title but female children will be a little harder.  All unmarried females have the title, "miss" regardless of age. Using the parch flag we can identify which females are with parents and assume
they are the children.  To keep things concise I'll display the average ages of the miss title for those with and without parents present.

In [11]:
print ("Avg age of 'Miss' Title", round(train_data[train_data.Title=="Miss"]['Age'].mean()))

print ("Avg age of 'Miss' Title travelling without Parents", round(train_data[(train_data.Title=="Miss") & (train_data.Parch==0)]['Age'].mean()))

print ("Avg age of 'Miss' Title travelling with Parents", round(train_data[(train_data.Title=="Miss") & (train_data.Parch!=0)]['Age'].mean()), '\n')

Avg age of 'Miss' Title 22
Avg age of 'Miss' Title travelling without Parents 28
Avg age of 'Miss' Title travelling with Parents 12 



Since the average age of female children is much different that the average age of female adults it's important to input missing values.  Before filling the missing values we should explore the embarked and fare values.

In [13]:
##Let us turn our attention to the missing fare
display(combined[combined.Fare.isnull()])

##Let us get fare per person
for df in [train_data, test_data, combined]:
    df['PeopleInTicket']=df['Ticket'].map(combined['Ticket'].value_counts())
    df['FarePerPerson']=df['Fare']/df['PeopleInTicket']

##Just take the mean fare for the PORT S and the Pclass & fill it. Remember to consider FarePerPerson and not Fare
print('Mean fare for this category: ', train_data[(train_data.Embarked=='S') & (train_data.Pclass==3)]['FarePerPerson'].mean())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
152,1044,NaN,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S,Mr


Mean fare for this category:  7.382647773383372


Filling in the missing fare value.  This time I will use the mean fare for 3rd class port s solo travellers.  In our EDA we saw that solo travellers sometimes had different fares

In [14]:
test_data.loc[test_data.Fare.isnull(), ['Fare','FarePerPerson']] = round(train_data[(train_data.Embarked=='S') & (train_data.Pclass==3) & (train_data.PeopleInTicket==1)]['Fare'].mean(),1)

Next comes the missing embarked rows.

In [15]:
display(combined[combined.Embarked.isnull()])

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,PeopleInTicket,FarePerPerson
61,62,1.0,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN,Miss,2,40.0
829,830,1.0,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN,Mrs,2,40.0


So each of these missing embarked valeus shares one similarity, they're both pclass 1 with 2 people in ticket and $40 fareperperson.  Lets check statistics grouped by the embarked class

In [17]:
train_data[(train_data.Pclass==1)].groupby('Embarked').agg({'FarePerPerson': 'mean', 'Fare': 'mean', 'PassengerId': 'count'})

,FarePerPerson,Fare,PassengerId
Embarked,,,
C,39.179223,104.718529,85
Q,30.000000,90.000000,2
S,30.211849,70.364862,127


In [18]:
#It seems obvious that port C is the correct answer here especially gien that only one group got on at port Q
train_data.Embarked.fillna('C', inplace=True)

Using the work from earlier we can fill in the missing age values now as well, remembering to use the Miss title carefully.  We can create a new category called FemaleChild

In [19]:
print(train_data.groupby(['Pclass','Sex','Title'])['Age'].agg({'mean', 'median', 'count'}))

for df in [train_data, test_data, combined]:
    df.loc[(df.Title=='Miss') & (df.Parch!=0) & (df.PeopleInTicket>1), 'Title']="FemaleChild"

display(combined[(combined.Age.isnull()) & (combined.Title=='FemaleChild')])

                       median       mean  count
Pclass Sex    Title                            
1      female Miss       30.0  29.744681     47
              Mrs        40.0  40.400000     35
              Royalty    48.0  43.333333      3
       male   Master      4.0   5.306667      3
              Mr         40.0  41.580460     87
              Officer    56.0  56.600000      5
              Royalty    42.0  42.166667      6
2      female Miss       24.0  22.390625     32
              Mrs        31.5  33.547619     42
       male   Master      1.0   2.258889      9
              Mr         31.0  32.768293     82
              Royalty    46.5  42.000000      8
3      female Miss       18.0  16.123188     69
              Mrs        31.0  33.515152     33
       male   Master      4.0   5.350833     24
              Mr         26.0  28.724891    229


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,PeopleInTicket,FarePerPerson
128,129,1.0,3,"Peter, Miss. Anna",female,NaN,1,1,2668,22.3583,F E69,C,FemaleChild,3,7.452767
180,181,0.0,3,"Sage, Miss. Constance Gladys",female,NaN,8,2,CA. 2343,69.5500,NaN,S,FemaleChild,11,6.322727
229,230,0.0,3,"Lefebre, Miss. Mathilde",female,NaN,3,1,4133,25.4667,NaN,S,FemaleChild,5,5.093340
409,410,0.0,3,"Lefebre, Miss. Ida",female,NaN,3,1,4133,25.4667,NaN,S,FemaleChild,5,5.093340
485,486,0.0,3,"Lefebre, Miss. Jeannie",female,NaN,3,1,4133,25.4667,NaN,S,FemaleChild,5,5.093340
792,793,0.0,3,"Sage, Miss. Stella Anna",female,NaN,8,2,CA. 2343,69.5500,NaN,S,FemaleChild,11,6.322727
863,864,0.0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.5500,NaN,S,FemaleChild,11,6.322727
888,889,0.0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,FemaleChild,4,5.862500
188,1080,NaN,3,"Sage, Miss. Ada",female,NaN,8,2,CA. 2343,69.5500,NaN,S,FemaleChild,11,6.322727


In [22]:
grp = train_data.groupby(['Pclass','Sex','Title'])['Age'].mean().reset_index()[['Sex', 'Pclass', 'Title', 'Age']]

#This is a lookup table that can tell you the mean age of a group when passed Pclass, sex, and title of that group.
print(grp)

       Sex  Pclass        Title        Age
0   female       1  FemaleChild  21.071429
1   female       1         Miss  33.424242
2   female       1          Mrs  40.400000
3   female       1      Royalty  43.333333
4     male       1       Master   5.306667
5     male       1           Mr  41.580460
6     male       1      Officer  56.600000
7     male       1      Royalty  42.166667
8   female       2  FemaleChild   9.916667
9   female       2         Miss  29.875000
10  female       2          Mrs  33.547619
11    male       2       Master   2.258889
12    male       2           Mr  32.768293
13    male       2      Royalty  42.000000
14  female       3  FemaleChild   6.500000
15  female       3         Miss  21.590909
16  female       3          Mrs  33.515152
17    male       3       Master   5.350833
18    male       3           Mr  28.724891


In [23]:
print('\n', 'For example: ', grp[(grp.Pclass==2) & (grp.Sex=='male') & (grp.Title=='Master')]['Age'].values[0])


 For example:  2.2588888888888885


In [24]:
##Define a function called fill_age. This will lookup the combination
##passed to it using above lookup table and return the value of the age associated
def fill_age(x):
    return grp[(grp.Pclass==x.Pclass)&(grp.Sex==x.Sex)&(grp.Title==x.Title)]['Age'].values[0]
##Here 'x' is the row containing the missing age. We look up the row's Pclass
##Sex and Title against the lookup table as shown previously and return the Age
##Now we have to call this fill_age function for every missing row for test, train

train_data['Age'], test_data['Age'] = [df.apply(lambda x: fill_age(x) if np.isnan(x['Age']) else x['Age'], axis=1) for df in [train_data, test_data]]

##End by combining the test and training data
combined=train_data.append(test_data)

This should result in a new set with all of our filled values based on our earlier findings.  Now that our dataset is clean all that is left is to group families and get modeling.